In [43]:
!python -V

Python 3.9.0


In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        return np.sqrt(mse)

In [46]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow/db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1750165638515, experiment_id='1', last_update_time=1750165638515, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

##### Prepare data for training and testing model

In [47]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [48]:
import os
os.getcwd()

'/workspaces/mlops_zoomcamp/02-experiment-tracking'

In [49]:
df_train = read_dataframe('data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('data/green_tripdata_2025-02.parquet')

In [50]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [51]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [52]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

#### Train model

In [53]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse(y_val, y_pred)

2025/06/17 14:40:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9204fc4a40b248299ad83e75ca3e4dfc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


np.float64(6.098889826481847)

##### Run experiment

In [54]:
with mlflow.start_run():
    alpha = 0.02

    mlflow.set_tag("developper", "Tsila")
    mlflow.log_param("train-data-path", "data/green_tripdata_2025-01.parquet")
    mlflow.log_param("validation-data-path", "data/green_tripdata_2025-02.parquet")
    mlflow.log_param("alpha",alpha)
    

    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)

    rmse_score = float(rmse(y_val, y_pred))
    mlflow.log_metric('rmse', rmse_score)

In [55]:
import pickle
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

##### Run experiment using hyperopt for hyperparameter optimization

In [56]:
import xgboost as xgb

In [57]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [58]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [59]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        
        rmse_score = float(rmse(y_val, y_pred))
        mlflow.log_metric('rmse', rmse_score)

    return {'loss': rmse_score, 'status': STATUS_OK}

In [60]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:40:43] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.94218                           
[1]	validation-rmse:6.05127                           
[2]	validation-rmse:5.70891                           
[3]	validation-rmse:5.57991                           
[4]	validation-rmse:5.51619                           
[5]	validation-rmse:5.49419                           
[6]	validation-rmse:5.47751                           
[7]	validation-rmse:5.47117                           
[8]	validation-rmse:5.46729                           
[9]	validation-rmse:5.46507                           
[10]	validation-rmse:5.46015                          
[11]	validation-rmse:5.45765                          
[12]	validation-rmse:5.45660                          
[13]	validation-rmse:5.45673                          
[14]	validation-rmse:5.45427                          
[15]	validation-rmse:5.45020                          
[16]	validation-rmse:5.45078                          
[17]	validation-rmse:5.45169                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:40:59] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.65298                                                    
[1]	validation-rmse:8.25308                                                    
[2]	validation-rmse:7.90148                                                    
[3]	validation-rmse:7.59328                                                    
[4]	validation-rmse:7.32404                                                    
[5]	validation-rmse:7.08952                                                    
[6]	validation-rmse:6.88604                                                    
[7]	validation-rmse:6.71043                                                    
[8]	validation-rmse:6.55776                                                    
[9]	validation-rmse:6.42601                                                    
[10]	validation-rmse:6.31232                                                   
[11]	validation-rmse:6.21461                                                   
[12]	validation-rmse:6.13060            

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:42:55] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:6.15990                                                    
[2]	validation-rmse:5.77286                                                    
[3]	validation-rmse:5.61338                                                    
[4]	validation-rmse:5.53741                                                    
[5]	validation-rmse:5.50190                                                    
[6]	validation-rmse:5.47961                                                    
[7]	validation-rmse:5.46493                                                    
[8]	validation-rmse:5.45768                                                    
[9]	validation-rmse:5.45055                                                    
[10]	validation-rmse:5.44097                                                   
[11]	validation-rmse:5.43678                                                   
[12]	validation-rmse:5.43283                                                   
[13]	validation-rmse:5.43056            

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:43:10] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.50233                                                    
[1]	validation-rmse:7.98625                                                    
  6%|▌         | 3/50 [02:30<39:17, 50.17s/trial, best loss: 5.338220368115263]


KeyboardInterrupt: 

In [61]:

mlflow.xgboost.autolog(disable=True)

##### Run experiment with best parameters

In [65]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
    "learning_rate" : 0.08225090750528806,
    "max_depth" : 19,
    "min_child_weight" : 2.087104744078911,
    "objective" : "reg:linear",
    "reg_alpha" : 0.022939506141609566,
    "reg_lambda" : 0.01229230260838323,
    "seed" : 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse_score = float(rmse(y_val, y_pred))
    mlflow.log_metric('rmse', rmse_score)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, name="models_mlflow")



/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:48:46] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.64947
[1]	validation-rmse:8.24427


[2]	validation-rmse:7.88752
[3]	validation-rmse:7.57292
[4]	validation-rmse:7.29697
[5]	validation-rmse:7.05257
[6]	validation-rmse:6.83894
[7]	validation-rmse:6.65540
[8]	validation-rmse:6.49261
[9]	validation-rmse:6.35391
[10]	validation-rmse:6.23142
[11]	validation-rmse:6.12732
[12]	validation-rmse:6.03369
[13]	validation-rmse:5.95414
[14]	validation-rmse:5.88533
[15]	validation-rmse:5.82420
[16]	validation-rmse:5.77196
[17]	validation-rmse:5.72627
[18]	validation-rmse:5.68739
[19]	validation-rmse:5.65299
[20]	validation-rmse:5.62340
[21]	validation-rmse:5.59771
[22]	validation-rmse:5.57511
[23]	validation-rmse:5.55400
[24]	validation-rmse:5.53647
[25]	validation-rmse:5.52074
[26]	validation-rmse:5.50745
[27]	validation-rmse:5.49470
[28]	validation-rmse:5.48321
[29]	validation-rmse:5.47402
[30]	validation-rmse:5.46591
[31]	validation-rmse:5.45722
[32]	validation-rmse:5.45024
[33]	validation-rmse:5.44360
[34]	validation-rmse:5.43736
[35]	validation-rmse:5.43221
[36]	validation-rmse:5

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:49:44] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/17 14:49:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


##### Run experiment using hyperopt for hyperparameter optimization and autolog

In [67]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "data/green_tripdata_2025-01.csv")
        mlflow.log_param("valid-data-path", "data/green_tripdata_2025-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse_score = float(rmse(y_val, y_pred))
        mlflow.log_metric('rmse', rmse_score)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
